In [ ]:
from main_functions import *
from sklearn.model_selection import train_test_split


""""DOWNLAOD DATA + CREATE PANDAS DATAFRAME"""

url="https://www.texaslottery.com/export/sites/lottery/Games/Powerball/Winning_Numbers/powerball.csv"
csv_path='raw_powerball.csv'

raw_df=url_to_df(url,csv_path)
pb_df=clean_df(raw_df)

""" SPLIT INTO TRAIN / VALIDATION/ TEST DATAFRAMES"""
train_df, test_df = train_test_split(pb_df, test_size=0.1, shuffle=False)
train_df, valid_df=train_test_split(train_df, test_size=0.2, shuffle=False)

 """NORMALIZE TRAIN / VALIDATION/ TEST DATAFRAMES"""

normalize='yes' #yes or no
if normalize=='yes':
    train_norm_df,scaler=norm_train_df(train_df)
    valid_norm_df=norm_other_df(valid_df,scaler)
    test_norm_df=norm_other_df(test_df,scaler)
else:
    train_norm_df=train_df
    valid_norm_df=valid_df
    test_norm_df=test_df

"""CREATE NUMPY ARRAYS OF TRAIN / VALIDATION / TEST FEATURES & LABELS 
(AKA INPUT FORMAT OF DEEP LEARNING MODEL)"""
window_len=10 #num of past games we'll consider for prediction
print('TRAIN DATA')
train_features, train_labels=DL_format(train_norm_df,window_len)
print('VALIDATION DATA')
valid_features, valid_labels=DL_format(valid_norm_df,window_len)
print('TEST DATA')
test_features, test_labels=DL_format(test_norm_df,window_len)

"""CREATE MODEL"""
model=create_model(train_features)

"""TRAIN MODEL aka model.fit(---)"""
#set hyperparameters
batch_size_num=15
epochs_num=10
val_split=0.2
trained_model=model.fit(train_features,train_labels,batch_size=batch_size_num,epochs=epochs_num,validation_data=(valid_features,valid_labels))

"""DIAGNOSE OVERFITTING/UNDERFITTING OF MODEL"""
plot_train_val(trained_model) 

eval_test(test_features,test_labels,normalize,scaler)